<a href="https://colab.research.google.com/github/h3it-dias/FastCamp_Agent_AI_CEIA/blob/main/Agent_ReAct_aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação do agente ReAct proposto pelo vídeo

## Lógica do ReAct

1.   Pensar: Compreender o que deve ser feito
2.   Agir: Escolher a melhor ação/tool e executar
3.   PAUSE: para aguardar o resultado
4.   Observação: Recebe o resultado

*   Caso a resposta ainda não foi fornecida, retornar para o "Pensar"

## Instalação do modelo Groq

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.6 MB/s eta 0:00:00


## Configuração da API key do groq

In [ ]:
import os
os.environ['GROQ_API_KEY'] = "sua_api_key_aqui"

## Fazendo conexão da Api com o Groq

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

## Construção da estrutura da Classe do Agente

In [ ]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    completion = client.chat.completions.create(
        messages=self.messages,
        model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

## Escrita do System Prompt

In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

## Criação das Tool/Ferramentas a serem usadas

In [ ]:
# tools
def calculate(operation):
  return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

## Execução manual do Agente

In [ ]:
neil_tyson = Agent(client, system_prompt)

In [ ]:
result = neil_tyson("What is the mass of Earth times 5?")
print(result)

In [ ]:
neil_tyson.messages

In [ ]:
result = neil_tyson()
print(result)

In [ ]:
observation =  get_planet_mass("Earth")
print(observation)

In [ ]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

In [ ]:
neil_tyson.messages

In [ ]:

result = neil_tyson()
print(result)

In [ ]:
observation = calculate("3.285e23 * 5")
print(observation)

In [ ]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

In [ ]:
neil_tyson.messages

## Construção do Loop para execução automática do Agente

In [ ]:
import re

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt)
  tools = ["calculate", "get_planet_mass"]
  next_prompt = query
  i=0
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      choise_tools = action[0][0]
      arg = action[0][1]

      if choise_tools in tools:
        result_tool = eval(f"{choise_tools}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation: Tool not found"

      print(next_prompt)
      continue

    if "Answer" in result:
      break

agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")